## Scripts for master's thesis
### Step 1. Preparing data

In [3]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import xmltodict, csv, graphviz
import pm4py
from datetime import datetime

### Reading input & extracting problems

In [4]:
# Reading input file
fd = open('external-43799.xml', 'r')
xml_file=fd.read()
fd.close()

soup=bs(xml_file, 'lxml')

# Preparing header for df_problems
cols = ["problem_title", "problem_name"]
rows = []

for tag in soup.findAll("problem"):
    title = tag["title"]
    name = tag["longname"]

    rows.append({"problem_title": title,
                 "problem_name": name})

df_problems = pd.DataFrame(rows, columns=cols)
df_problems = df_problems.set_index('problem_name')

df_problems

,problem_title
problem_name,
Дележ яблок-1,A
Последняя цифра,B
100A summer,C
100 раз подряд в квадрате,D
Сумма цифр трехзначного числа,E
Пингвины,F*
Шашки,G*
Исполнитель раздвоитель,H*
Ханойские башни,I**


### Extracting users

In [5]:
cols = ["user_id", "login", "user_name"]
rows = []

for tag in soup.findAll("user"):
    id = tag["id"]
    login = tag["loginname"]
    name = tag["displayedname"]

    rows.append({"user_id": id,
                 "login": login,
                 "user_name": name})

df_users = pd.DataFrame(rows, columns=cols)
df_users =df_users.set_index('user_id')

df_users

,login,user_name
user_id,,
104786219,samonenko,Илья Самоненко
104786220,hse-cs-fd-2022-1,hse-cs-fd-2022-1
104786221,hse-cs-fd-2022-2,hse-cs-fd-2022-2
104786222,hse-cs-fd-2022-3,hse-cs-fd-2022-3
104786223,hse-cs-fd-2022-4,hse-cs-fd-2022-4
...,...,...
104786315,hse-cs-fd-2022-96,hse-cs-fd-2022-96
104786316,hse-cs-fd-2022-97,hse-cs-fd-2022-97
104786317,hse-cs-fd-2022-98,hse-cs-fd-2022-98


### Extracting submits without tests and tests

In [6]:
# Preparing header for submits df
cols = ["contestTime", "absoluteTime", "lastUpdateTime", "id", "problem", "user", "lang", "maxTime", "maxMemory", "verdict"]
rows = []

# Preparing header for submit's tests df_tests
test_cols = ["submit_id", "number", "verdict", "time", "memory"]
test_rows = []

# Parsing input for submits (without tests)
for tag in soup.findAll("submit"):
    contestTime = tag["contesttime"]
    absoluteTime = tag["absolutetime"]
    lastUpdateTime = tag["lastupdatetime"]
    id = tag["id"]
    problem = tag["problemtitle"]
    user = tag["userid"]
    lang = tag["languageid"]
    time = tag["maxtimeusedmillis"]
    memory = tag["maxmemoryusedbytes"]
    verdict = tag["verdict"]
    
    for test in tag.findAll("test"):
        number = test["number"]
        t_verdict = test["verdict"]
        t_time = test["timeusedmillis"]
        t_memory = test["memoryusedbytes"]

        test_rows.append({"submit_id": id,
                          "number": number,
                          "verdict": t_verdict,
                          "time": t_time,
                          "memory": t_memory})
    
    rows.append({"contestTime": contestTime,
                 "absoluteTime": absoluteTime[:10],
                 "lastUpdateTime": lastUpdateTime[:10],
                 "id": id,
                 "problem": problem,
                 "user": user,
                 "lang": lang,
                 "maxTime": time,
                 "maxMemory": memory,
                 "verdict": verdict})
    
df = pd.DataFrame(rows, columns = cols)

df_tests = pd.DataFrame(test_rows, columns=test_cols)
df_tests = df_tests.set_index('submit_id')

df

,contestTime,absoluteTime,lastUpdateTime,id,problem,user,lang,maxTime,maxMemory,verdict
0,27347772,1670474147,1670474150,78966875,A,104786269,gcc7_3,0,0,CE
1,27357271,1670474157,1670474167,78966877,A,104786269,python3_7_3,253,4448256,OK
2,27415883,1670474215,1670474232,78966881,B,104786269,python3_7_3,70,4448256,OK
3,27482637,1670474282,1670474285,78966890,C,104786269,python3_7_3,47,4083712,OK
4,27571229,1670474371,1670474378,78966899,D,104786269,python3_7_3,48,4444160,OK
...,...,...,...,...,...,...,...,...,...,...
609,137125694,1670583925,1670583928,79063954,G*,104786246,python3_7_3,46,3928064,WA
610,137289823,1670584089,1670584104,79064135,G*,104786246,python3_7_3,79,4448256,OK
611,144321452,1670591121,1670591132,79073077,I**,104786301,python3_7_3,167,4444160,OK
612,146125154,1670592925,1670592929,79075929,J**,104786301,python3_7_3,51,4202496,WA


### Step 2. Transforming dataframe to event log

In [9]:
# converting absoluteTime from timestamp to datetime format
for i in range(614):
    df.iloc[i].absoluteTime = datetime.fromtimestamp(int(df.iloc[i].absoluteTime))

df

,contestTime,absoluteTime,lastUpdateTime,id,problem,user,lang,maxTime,maxMemory,verdict
0,27347772,2022-12-08 07:35:47,1670474150,78966875,A,104786269,gcc7_3,0,0,CE
1,27357271,2022-12-08 07:35:57,1670474167,78966877,A,104786269,python3_7_3,253,4448256,OK
2,27415883,2022-12-08 07:36:55,1670474232,78966881,B,104786269,python3_7_3,70,4448256,OK
3,27482637,2022-12-08 07:38:02,1670474285,78966890,C,104786269,python3_7_3,47,4083712,OK
4,27571229,2022-12-08 07:39:31,1670474378,78966899,D,104786269,python3_7_3,48,4444160,OK
...,...,...,...,...,...,...,...,...,...,...
609,137125694,2022-12-09 14:05:25,1670583928,79063954,G*,104786246,python3_7_3,46,3928064,WA
610,137289823,2022-12-09 14:08:09,1670584104,79064135,G*,104786246,python3_7_3,79,4448256,OK
611,144321452,2022-12-09 16:05:21,1670591132,79073077,I**,104786301,python3_7_3,167,4444160,OK
612,146125154,2022-12-09 16:35:25,1670592929,79075929,J**,104786301,python3_7_3,51,4202496,WA


In [8]:
format_df = df[['absoluteTime', 'problem', 'user', 'lang', 'maxTime', 'maxMemory', 'verdict']].copy()
format_df.problem = format_df.problem + ' ' + format_df.verdict
format_df = format_df.rename(columns = {'lang':'case:lang', 'maxTime':'case:maxTime', 'maxMemory':'case:maxMemory'})
format_df = pm4py.format_dataframe(format_df, case_id = 'user', activity_key = 'problem', timestamp_key='absoluteTime')
event_log = pm4py.convert_to_event_log(format_df)

pm4py.write_xes(event_log, 'log.xes')
event_log

C:\Users\anastasiya.alekseeva\AppData\Roaming\Python\Python310\site-packages\pm4py\utils.py:494: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [8]:
log_start = pm4py.get_start_activities(event_log)
# log_start

In [9]:
log_end = pm4py.get_end_activities(event_log)
# log_end

In [10]:
variants = pm4py.get_variants_as_tuples(event_log)
#variants